In [1]:
import tensorflow as tf
import numpy as np
from utils import *
import os
import pickle
from cifar10_input import *
from cifar10_models import *
import logging

tf.logging.set_verbosity(tf.logging.INFO)

# get TF logger
log = logging.getLogger('tensorflow')
log.setLevel(logging.INFO)

# create formatter and add it to the handlers
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')

# create file handler which logs even debug messages
fh = logging.FileHandler('tensorflow_cifar10_discriminator.log')
fh.setLevel(logging.INFO)
fh.setFormatter(formatter)
log.addHandler(fh)

#### SET THIS BEFORE RUNNING
MODEL_DIR = "cifar10/"
MODEL_NAME = "cifar10_scm_{}_{}_{}_{}" #wd1_wd2_wd3_drop1
MODEL_PATH = os.path.join(MODEL_DIR, MODEL_NAME)
DISCRIMINATOR_DIR = "discriminators/{}/" #model_checkpoint
DISCRIMINATOR_NAME = "discriminator_{}_{}" #drop1_drop2
DISCRIMINATOR_PATH = os.path.join(DISCRIMINATOR_DIR, DISCRIMINATOR_NAME)

class Namespace(object): pass

class BundledData(InitializableMixin):
    def __init__(self, train_in, train_out, test_in, test_out):
        with tf.name_scope("data_splitter"):
            self.use_test_data = tf.Variable(0, name="input_test_vs_train", trainable=False)
            use_test_data = tf.cast(self.use_test_data, tf.bool)
            self.input = tf.cond(use_test_data, lambda: test_in, lambda: train_in)
            self.input.set_shape(test_in.shape)
            self.output = tf.cond(use_test_data, lambda: test_out, lambda: train_out)
            self.output.set_shape(test_out.shape)

In [2]:
def train(model, discriminator, data, name, 
          model_restore_point = None, discriminator_restore_point = None, 
          drop1=1., drop2=1., save=True, steps=500):
    with tf.Session() as sess:
        model._restore_or_init(sess, model_restore_point)
        model.drop_fc1.assign(1.)
        data._initialize(sess)
        discriminator._restore_or_init(sess, discriminator_restore_point)
        tf.assign(discriminator.dropout_rate_002, drop1)
        tf.assign(discriminator.dropout_rate_001, drop2)
    
        for step in range(steps):
            sess.run(discriminator.train_step)
            global_step = discriminator.global_step.eval()
            use_test_data = global_step%2
            data.use_test_data.assign(use_test_data)
            discriminator.use_test_data.assign(use_test_data)
        if save:
            checkpoint_name = CHECKPOINT.format(name) + "_" + str(global_step).rjust(10,"0") 
            print("Saving to: ", checkpoint_name)
            discriminator.saver.save(sess, checkpoint_name)
        
    return checkpoint_name
        
def evaluate(model, discriminator, test_data_bool, 
             model_restore_point = None, discriminator_restore_point = None):
    acc = []
    with tf.Session() as sess:
        model._restore_or_init(sess, model_restore_point)
        model.drop_fc1.assign(1.)
        discriminator._restore_or_init(sess, discriminator_restore_point)
        discriminator.dropout_rate_001.assign(1.)
        discriminator.dropout_rate_002.assign(1.)
        discriminator.use_test_data.assign(test_data_bool)
        try:
            while True:
                acc.append(discriminator.accuracy.eval())
        except tf.errors.OutOfRangeError as e:
            pass
    return np.mean(acc)

@np.vectorize
def nplte(x,y):
    return x<=y

def train_to_convergence(model_restore_point, train_files, test_files, drop1=1., drop2=1., save_rate=10):
    BATCH_SIZE = 100
    train_acc = []
    test_acc = []
    name = DISCRIMINATOR_PATH.format(os.path.join(*model_restore_point.split("/")[2:]), drop1, drop2)
    make_dirs(ROOT.format(name))
    checkpoint = latest_checkpoint(name)
    discriminator_training = Namespace()
    discriminator_training.graph = tf.Graph()
    with discriminator_training.graph.as_default():
        dt = discriminator_training
        dt.train_data = Cifar10Record(train_files, batch_size=BATCH_SIZE)
        dt.test_data = Cifar10Record(test_files, batch_size=BATCH_SIZE)
        dt.data = BundledData(dt.train_data.image, dt.train_data.label, 
                              dt.test_data.image, dt.test_data.label)
        dt.model = Cifar10ShallowConvolutionalModel(dt.data.input, dt.data.output)
        dt.discriminator = Discriminator_Output(dt.model, BATCH_SIZE)
    while (
        (len(train_acc)<30 or nplte(np.array(train_acc[-20:]) - np.array(train_acc[-21:-1]),0).sum() < 11)
        and
        (len(test_acc)<30 or nplte(np.array(test_acc[-20:]) - np.array(test_acc[-21:-1]),0).sum() < 11)
    ): # until the accuracy is just as likely to go down as up
            
        tf.reset_default_graph()
        train_data = Cifar10Record(train_files, batch_size=BATCH_SIZE, epochs=1)
        model = Cifar10ShallowConvolutionalModel(train_data.image, train_data.label)
        discriminator = Discriminator_Output(model, BATCH_SIZE)
        test_data_bool = 0
        train_acc.append(evaluate(model, discriminator, test_data_bool, model_restore_point = model_restore_point,
                                  discriminator_restore_point = checkpoint))
        
        tf.reset_default_graph()
        test_data = Cifar10Record(test_files, batch_size=BATCH_SIZE, epochs=1)
        model = Cifar10ShallowConvolutionalModel(test_data.image, test_data.label)
        discriminator = Discriminator_Output(model, BATCH_SIZE)
        test_data_bool = 1
        test_acc.append(evaluate(model, discriminator, test_data_bool, model_restore_point = model_restore_point, 
                                  discriminator_restore_point = checkpoint))
        
        with discriminator_training.graph.as_default():
            checkpoint = train(discriminator_training.model, discriminator_training.discriminator, discriminator_training.data,
                 name, model_restore_point = model_restore_point, discriminator_restore_point = checkpoint,
                 drop1 = drop1, drop2 = drop2, save = True)
        
        tf.logging.info(train_acc[-21:])
        tf.logging.info(test_acc[-21:])
        
    return train_acc, test_acc

In [3]:
class Discriminator_OutputLabel(TrainableMixin):
    def __init__(self, model, batch_size, summaries=True): 
        self.model = model
        self.batch_size = batch_size
        self._init_graph(summaries)
        self._init_saver()

    def _init_graph(self, summaries=True):
        with tf.name_scope("discriminator"):
            
            with tf.name_scope("meta_variables"):
                self.global_step = tf.Variable(0, name="global_step", trainable=False)
                self.learning_rate = tf.Variable(0.001, name="learning_rate", trainable=False)

            with tf.name_scope("layer_000_input"):
                self.input_dim = list(np.multiply(2, self.model.output_dim))
                self.output_dim = [2]
                self.use_test_data = tf.Variable(1, trainable=False)
                self.y_ = tf.one_hot(self.use_test_data, 2, on_value=1.0, off_value=0.0)
                self.y_ = tf.reshape(self.y_, [1] + self.output_dim)
                self.y_ = tf.tile(self.y_,[self.batch_size,1])
                self.x = tf.concat((self.model.proba, self.model.y_),1)
                layer_000_out_dim = self.input_dim
                
            with tf.name_scope("layer_001_fc"):
                layer_001_in_dim = layer_000_out_dim
                layer_001_out_dim = [100]
                self.W_001 = tf.Variable(tf.random_normal(layer_001_in_dim + layer_001_out_dim,0,0.1), name="weights")
                self.b_001 = tf.Variable(tf.random_normal(layer_001_out_dim,0,0.1), name="bias")
                self.logits_001 = tf.add(tf.matmul(self.x,self.W_001, name="mul_weights"), self.b_001, name="add_bias")
                self.bent_001 = tf.nn.relu(self.logits_001, name="relu")
                self.dropout_rate_001 = tf.Variable(1.0, name="dropout_rate", trainable=False)
                self.dropout_001 = tf.nn.dropout(self.bent_001, self.dropout_rate_001, name="dropout")
                
            with tf.name_scope("layer_002_fc"):
                layer_002_in_dim = layer_001_out_dim
                layer_002_out_dim = [10]
                self.W_002 = tf.Variable(tf.random_normal(layer_002_in_dim + layer_002_out_dim,0,0.1), name="weights")
                self.b_002 = tf.Variable(tf.random_normal(layer_002_out_dim,0,0.1), name="bias")
                self.logits_002 = tf.add(tf.matmul(self.dropout_001,self.W_002, name="mul_weights"), self.b_002, name="add_bias")
                self.bent_002 = tf.nn.relu(self.logits_002, name="relu")
                self.dropout_rate_002 = tf.Variable(1.0, name="dropout_rate", trainable=False)
                self.dropout_002 = tf.nn.dropout(self.bent_002, self.dropout_rate_002, name="dropout")

            with tf.name_scope("layer_003_fc"):
                layer_003_in_dim = layer_002_out_dim
                layer_003_out_dim = self.output_dim
                self.W_003 = tf.Variable(tf.random_normal(layer_003_in_dim + layer_003_out_dim,0,0.1), name="weights")
                self.b_003 = tf.Variable(tf.random_normal(layer_003_out_dim,0,0.1), name="bias")
                self.logits_003 = tf.add(tf.matmul(self.dropout_002,self.W_003, name="mul_weights"), self.b_003, name="add_bias")
                
            with tf.name_scope("outputs"):
                self.logits = self.logits_003
                self.proba = tf.nn.softmax(self.logits)
                self.loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=self.y_, logits=self.logits))
                
            with tf.name_scope("summaries"):
                self.correct_prediction = tf.equal(tf.argmax(self.logits,1), tf.argmax(self.y_,1))
                self.accuracy = tf.reduce_mean(tf.cast(self.correct_prediction, tf.float32), name="accuracy")
                self.summaries = {}
                if summaries:
                    self.summaries = {
                        "batch acc.": tf.summary.scalar("batch_acc", self.accuracy)
                    }

            with tf.name_scope("training"): 
                self.optimizer = tf.train.GradientDescentOptimizer(self.learning_rate, name="optimizer")
                self.grads = self.optimizer.compute_gradients(self.loss)
                self.grad_application = self.optimizer.apply_gradients(self.grads, global_step = self.global_step)
                with tf.control_dependencies([self.grad_application] + list(self.summaries.values())):
                    self.train_step = tf.no_op(name="train_step")


In [4]:
class Discriminator_Output(TrainableMixin):
    def __init__(self, model, batch_size, summaries=True): 
        self.model = model
        self.batch_size = batch_size
        self._init_graph(summaries)
        self._init_saver()

    def _init_graph(self, summaries=True):
        with tf.name_scope("discriminator"):
            
            with tf.name_scope("meta_variables"):
                self.global_step = tf.Variable(0, name="global_step", trainable=False)
                self.learning_rate = tf.Variable(0.001, name="learning_rate", trainable=False)

            with tf.name_scope("layer_000_input"):
                self.input_dim = self.model.output_dim
                self.output_dim = [2]
                self.use_test_data = tf.Variable(1, trainable=False)
                self.y_ = tf.one_hot(self.use_test_data, 2, on_value=1.0, off_value=0.0)
                self.y_ = tf.reshape(self.y_, [1] + self.output_dim)
                self.y_ = tf.tile(self.y_,[self.batch_size,1])
                self.x = self.model.proba
                layer_000_out_dim = self.input_dim
                
            with tf.name_scope("layer_001_fc"):
                layer_001_in_dim = layer_000_out_dim
                layer_001_out_dim = [100]
                self.W_001 = tf.Variable(tf.random_normal(layer_001_in_dim + layer_001_out_dim,0,0.1), name="weights")
                self.b_001 = tf.Variable(tf.random_normal(layer_001_out_dim,0,0.1), name="bias")
                self.logits_001 = tf.add(tf.matmul(self.x,self.W_001, name="mul_weights"), self.b_001, name="add_bias")
                self.bent_001 = tf.nn.relu(self.logits_001, name="relu")
                self.dropout_rate_001 = tf.Variable(1.0, name="dropout_rate", trainable=False)
                self.dropout_001 = tf.nn.dropout(self.bent_001, self.dropout_rate_001, name="dropout")
                
            with tf.name_scope("layer_002_fc"):
                layer_002_in_dim = layer_001_out_dim
                layer_002_out_dim = [10]
                self.W_002 = tf.Variable(tf.random_normal(layer_002_in_dim + layer_002_out_dim,0,0.1), name="weights")
                self.b_002 = tf.Variable(tf.random_normal(layer_002_out_dim,0,0.1), name="bias")
                self.logits_002 = tf.add(tf.matmul(self.dropout_001,self.W_002, name="mul_weights"), self.b_002, name="add_bias")
                self.bent_002 = tf.nn.relu(self.logits_002, name="relu")
                self.dropout_rate_002 = tf.Variable(1.0, name="dropout_rate", trainable=False)
                self.dropout_002 = tf.nn.dropout(self.bent_002, self.dropout_rate_002, name="dropout")

            with tf.name_scope("layer_003_fc"):
                layer_003_in_dim = layer_002_out_dim
                layer_003_out_dim = self.output_dim
                self.W_003 = tf.Variable(tf.random_normal(layer_003_in_dim + layer_003_out_dim,0,0.1), name="weights")
                self.b_003 = tf.Variable(tf.random_normal(layer_003_out_dim,0,0.1), name="bias")
                self.logits_003 = tf.add(tf.matmul(self.dropout_002,self.W_003, name="mul_weights"), self.b_003, name="add_bias")
                
            with tf.name_scope("outputs"):
                self.logits = self.logits_003
                self.proba = tf.nn.softmax(self.logits)
                self.loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=self.y_, logits=self.logits))
                
            with tf.name_scope("summaries"):
                self.correct_prediction = tf.equal(tf.argmax(self.logits,1), tf.argmax(self.y_,1))
                self.accuracy = tf.reduce_mean(tf.cast(self.correct_prediction, tf.float32), name="accuracy")
                self.summaries = {}
                if summaries:
                    self.summaries = {
                        "batch acc.": tf.summary.scalar("batch_acc", self.accuracy)
                    }

            with tf.name_scope("training"): 
                self.optimizer = tf.train.GradientDescentOptimizer(self.learning_rate, name="optimizer")
                self.grads = self.optimizer.compute_gradients(self.loss)
                self.grad_application = self.optimizer.apply_gradients(self.grads, global_step = self.global_step)
                with tf.control_dependencies([self.grad_application] + list(self.summaries.values())):
                    self.train_step = tf.no_op(name="train_step")

In [5]:
model_path = MODEL_PATH.format(0.0,0.0,0.0,0.2)
print(model_path)
model_restore_point = latest_checkpoint(model_path)
print(model_restore_point)

cifar10/cifar10_scm_0.0_0.0_0.0_0.2
../models/cifar10/cifar10_scm_0.0_0.0_0.0_0.2/model.ckpt_0000153600


In [7]:
train_files, test_files, validation_files = maybe_download_and_extract()
train_to_convergence(model_restore_point, train_files, test_files, drop1=0.3, drop2=0.3)

INFO:tensorflow:Restoring parameters from ../models/cifar10/cifar10_scm_0.0_0.0_0.0_0.2/model.ckpt_0000153600
INFO:tensorflow:Restoring parameters from ../models/cifar10/cifar10_scm_0.0_0.0_0.0_0.2/model.ckpt_0000153600
INFO:tensorflow:Restoring parameters from ../models/cifar10/cifar10_scm_0.0_0.0_0.0_0.2/model.ckpt_0000153600
Saving to:  ../models/discriminators/cifar10/cifar10_scm_0.0_0.0_0.0_0.2/model.ckpt_0000153600/discriminator_0.3_0.3/model.ckpt_0000000500
INFO:tensorflow:../models/discriminators/cifar10/cifar10_scm_0.0_0.0_0.0_0.2/model.ckpt_0000153600/discriminator_0.3_0.3/model.ckpt_0000000500 is not in all_model_checkpoint_paths. Manually adding it.
INFO:tensorflow:[1.0]
INFO:tensorflow:[0.0]
INFO:tensorflow:Restoring parameters from ../models/cifar10/cifar10_scm_0.0_0.0_0.0_0.2/model.ckpt_0000153600
INFO:tensorflow:Restoring parameters from ../models/discriminators/cifar10/cifar10_scm_0.0_0.0_0.0_0.2/model.ckpt_0000153600/discriminator_0.3_0.3/model.ckpt_0000000500
INFO:te

INFO:tensorflow:[0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]
INFO:tensorflow:Restoring parameters from ../models/cifar10/cifar10_scm_0.0_0.0_0.0_0.2/model.ckpt_0000153600
INFO:tensorflow:Restoring parameters from ../models/discriminators/cifar10/cifar10_scm_0.0_0.0_0.0_0.2/model.ckpt_0000153600/discriminator_0.3_0.3/model.ckpt_0000003500
INFO:tensorflow:Restoring parameters from ../models/cifar10/cifar10_scm_0.0_0.0_0.0_0.2/model.ckpt_0000153600
INFO:tensorflow:Restoring parameters from ../models/discriminators/cifar10/cifar10_scm_0.0_0.0_0.0_0.2/model.ckpt_0000153600/discriminator_0.3_0.3/model.ckpt_0000003500
INFO:tensorflow:Restoring parameters from ../models/cifar10/cifar10_scm_0.0_0.0_0.0_0.2/model.ckpt_0000153600
INFO:tensorflow:Restoring parameters from ../models/discriminators/cifar10/cifar10_scm_0.0_0.0_0.0_0.2/model.ckpt_0000153600/discriminator_0.3_0.3/model.ckpt_0000003500
Saving to:  ../models/discriminators/cifar10/cifar10_scm_0.0_0.0_0.0_0.2/model.ckpt_0000153600/discriminator_0.

INFO:tensorflow:Restoring parameters from ../models/cifar10/cifar10_scm_0.0_0.0_0.0_0.2/model.ckpt_0000153600
INFO:tensorflow:Restoring parameters from ../models/discriminators/cifar10/cifar10_scm_0.0_0.0_0.0_0.2/model.ckpt_0000153600/discriminator_0.3_0.3/model.ckpt_0000006500
INFO:tensorflow:Restoring parameters from ../models/cifar10/cifar10_scm_0.0_0.0_0.0_0.2/model.ckpt_0000153600
INFO:tensorflow:Restoring parameters from ../models/discriminators/cifar10/cifar10_scm_0.0_0.0_0.0_0.2/model.ckpt_0000153600/discriminator_0.3_0.3/model.ckpt_0000006500
Saving to:  ../models/discriminators/cifar10/cifar10_scm_0.0_0.0_0.0_0.2/model.ckpt_0000153600/discriminator_0.3_0.3/model.ckpt_0000007000
INFO:tensorflow:../models/discriminators/cifar10/cifar10_scm_0.0_0.0_0.0_0.2/model.ckpt_0000153600/discriminator_0.3_0.3/model.ckpt_0000007000 is not in all_model_checkpoint_paths. Manually adding it.
INFO:tensorflow:[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]
INFO:tensorflow

INFO:tensorflow:Restoring parameters from ../models/cifar10/cifar10_scm_0.0_0.0_0.0_0.2/model.ckpt_0000153600
INFO:tensorflow:Restoring parameters from ../models/discriminators/cifar10/cifar10_scm_0.0_0.0_0.0_0.2/model.ckpt_0000153600/discriminator_0.3_0.3/model.ckpt_0000009500
INFO:tensorflow:Restoring parameters from ../models/cifar10/cifar10_scm_0.0_0.0_0.0_0.2/model.ckpt_0000153600
INFO:tensorflow:Restoring parameters from ../models/discriminators/cifar10/cifar10_scm_0.0_0.0_0.0_0.2/model.ckpt_0000153600/discriminator_0.3_0.3/model.ckpt_0000009500
Saving to:  ../models/discriminators/cifar10/cifar10_scm_0.0_0.0_0.0_0.2/model.ckpt_0000153600/discriminator_0.3_0.3/model.ckpt_0000010000
INFO:tensorflow:../models/discriminators/cifar10/cifar10_scm_0.0_0.0_0.0_0.2/model.ckpt_0000153600/discriminator_0.3_0.3/model.ckpt_0000010000 is not in all_model_checkpoint_paths. Manually adding it.
INFO:tensorflow:[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 

INFO:tensorflow:Restoring parameters from ../models/cifar10/cifar10_scm_0.0_0.0_0.0_0.2/model.ckpt_0000153600
INFO:tensorflow:Restoring parameters from ../models/discriminators/cifar10/cifar10_scm_0.0_0.0_0.0_0.2/model.ckpt_0000153600/discriminator_0.3_0.3/model.ckpt_0000012500
INFO:tensorflow:Restoring parameters from ../models/cifar10/cifar10_scm_0.0_0.0_0.0_0.2/model.ckpt_0000153600
INFO:tensorflow:Restoring parameters from ../models/discriminators/cifar10/cifar10_scm_0.0_0.0_0.0_0.2/model.ckpt_0000153600/discriminator_0.3_0.3/model.ckpt_0000012500
INFO:tensorflow:Restoring parameters from ../models/cifar10/cifar10_scm_0.0_0.0_0.0_0.2/model.ckpt_0000153600
INFO:tensorflow:Restoring parameters from ../models/discriminators/cifar10/cifar10_scm_0.0_0.0_0.0_0.2/model.ckpt_0000153600/discriminator_0.3_0.3/model.ckpt_0000012500
Saving to:  ../models/discriminators/cifar10/cifar10_scm_0.0_0.0_0.0_0.2/model.ckpt_0000153600/discriminator_0.3_0.3/model.ckpt_0000013000
INFO:tensorflow:../model

([1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0],
 [0.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0])